In [22]:
import pandas as pd
import re
import numpy as np
from scipy.stats import norm

# 분석할 데이터 파일을 불러옵니다.
file_path = 'final_sentiment_analysis_results.csv'
try:
    eco_final_df = pd.read_csv(file_path)
    print("✅ 데이터 파일을 성공적으로 불러왔습니다.")
    print(f"데이터 형태: {eco_final_df.shape}")
    print("\n[데이터 미리보기]")
    display(eco_final_df.head())
except FileNotFoundError:
    print(f"❗️ 오류: '{file_path}' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")

✅ 데이터 파일을 성공적으로 불러왔습니다.
데이터 형태: (176, 8)

[데이터 미리보기]


,Park_nm,Topic,Count,Name,Representation,Representative_Docs,sentiment_label,sentiment_score
0,용산_효창공원,1,64,1_나무_김구_기념관_효창,"['나무', '김구', '기념관', '효창', '선생', '오래', '역사', '시...","['숲도 좋고 운동하기좋은곳 역사가 있고(윤봉길) 김구선생님 기념관도 있고', '도...",긍정,0.598557
1,용산_효창공원,2,43,2_역사_심속_의미_오래,"['역사', '심속', '의미', '오래', '여유', '휴식', '집기', '벌레...","['걷기와 휴식 역사가 있는 편안한 공원', '역사적 의미가 있는 공원', '여름철...",긍정,0.971144
2,용산_효창공원,4,16,4_방문_아름답_시내_최고,"['방문', '아름답', '시내', '최고', '한국', '조성', '괜찮', '추...",['아름답고 잘 관리된 공원 그런데 제가 서울에서 방문한 모든 공원은 완벽하게 관리...,긍정,0.616219
3,종로_낙산공원,0,581,0_올라가_데이트_사람_예쁘,"['올라가', '데이트', '사람', '예쁘', '경치', '힘들', '주차', '...",['정상에 올라가면 나름 운치있고 성곽도 있고ㅎㅎ 예쁨^^ 사진찍기좋음ㅎㅎ 데이트 ...,긍정,0.824299
4,종로_낙산공원,1,180,1_카페_도심_올라가_주차장,"['카페', '도심', '올라가', '주차장', '경치', '대학', '바라보', ...",['걸어서 성곽길을 걷다보면 아름다운 카페들도 나오고 운치있는곳들이 너무많은곳 연인...,긍정,0.942347


In [23]:
# 1. 생태/경관 관련 키워드 정의
# 수동으로 뽑아냈어요~^^
eco_keywords = [
    '나무', '자연', '아름답', '공기', '산책로', '정원', '단풍', '벚꽃',
    '식물', '산책', '전망', '전망대', '호수', '동물', '야경', '경치',
    '소풍', '향기', '장미', '근린공원', '조깅', '분수', '잔디밭', '고요',
    '바위', '잔디', '생태', '하늘', '축제', '햇살', '등산', '조망',
    '나들이', '연못', '조경', '둘레길', '숲길', '저수지', '가을', '산림', '날씨',
    '풍경', '정취', '꽃구경', '경관', '예쁘', '여름', '바라보', '명소',
    '운치', '그늘', '조명', '명산', '물소리', '일출', '꽃피', '수국',
    '개나리', '겨울', '모래', '습지', '수목원', '코스모스', '너구리', '강아지', '개구리',
    '잉어', '식물원', '물고기', '허브', '나뭇잎', '나무숲', '초록', '계곡',
    '암벽', '강물', '녹지', '오솔길', '호숫가', '사슴', '토끼', '새싹', '목련', '꽃사슴',
    '봄꽃', '광장', '만개', '정상', '환경', '아침', '저녁', '탁트인', '아름다우',
    '어우러지', '흐드러지', '피어나', '천지', '전경', '산행', '조화',
    '달빛', '선선', '휴식처', '한가로이', '야경', '강과'
    ]

# str.contains()와 정규 표현식'|'.join()을 사용하여 키워드 중 하나라도 포함되면 선택
keyword_pattern = '|'.join(eco_keywords)
eco_final_df['topic_counts'] = eco_final_df['Name'].apply(
    lambda x: len(re.findall(keyword_pattern, str(x)))
)
print(f"생태/경관 관련 주제 추가 완료")
display(eco_final_df.head(20))

생태/경관 관련 주제 추가 완료


,Park_nm,Topic,Count,Name,Representation,Representative_Docs,sentiment_label,sentiment_score,topic_counts
0,용산_효창공원,1,64,1_나무_김구_기념관_효창,"['나무', '김구', '기념관', '효창', '선생', '오래', '역사', '시...","['숲도 좋고 운동하기좋은곳 역사가 있고(윤봉길) 김구선생님 기념관도 있고', '도...",긍정,0.598557,1
1,용산_효창공원,2,43,2_역사_심속_의미_오래,"['역사', '심속', '의미', '오래', '여유', '휴식', '집기', '벌레...","['걷기와 휴식 역사가 있는 편안한 공원', '역사적 의미가 있는 공원', '여름철...",긍정,0.971144,0
2,용산_효창공원,4,16,4_방문_아름답_시내_최고,"['방문', '아름답', '시내', '최고', '한국', '조성', '괜찮', '추...",['아름답고 잘 관리된 공원 그런데 제가 서울에서 방문한 모든 공원은 완벽하게 관리...,긍정,0.616219,1
3,종로_낙산공원,0,581,0_올라가_데이트_사람_예쁘,"['올라가', '데이트', '사람', '예쁘', '경치', '힘들', '주차', '...",['정상에 올라가면 나름 운치있고 성곽도 있고ㅎㅎ 예쁨^^ 사진찍기좋음ㅎㅎ 데이트 ...,긍정,0.824299,1
4,종로_낙산공원,1,180,1_카페_도심_올라가_주차장,"['카페', '도심', '올라가', '주차장', '경치', '대학', '바라보', ...",['걸어서 성곽길을 걷다보면 아름다운 카페들도 나오고 운치있는곳들이 너무많은곳 연인...,긍정,0.942347,0
5,종로_낙산공원,2,23,2_종로_출구_거리_둘레길,"['종로', '출구', '거리', '둘레길', '바라보', '올라오', '저녁', ...",['동묘역 10번출구에서 종로03마을버스타면 숨안차고 오를수 있는 낙산공원^^ 사계...,긍정,0.814306,1
6,종로_낙산공원,3,20,3_전망대_언덕_혜화문_대학,"['전망대', '언덕', '혜화문', '대학', '벽화', '정상', '성지', '...","['이화동벽화마을과 같이 있어 좋음 남산과 북한산이 보이는 전망대', '주로 산책길...",중립,0.644720,1
7,마포_경의선숲길공원,2,63,2_아름답_예쁘_낭만_가볍,"['아름답', '예쁘', '낭만', '가볍', '도시', '사진', '기분', '다...","['아름답고 상쾌한 곳', '아름답고 낭만이 있는곳', '도시 한가운데서 산책하고 ...",긍정,0.989085,2
8,마포_경의선숲길공원,3,42,3_아침_장소_데이트_산책로,"['아침', '장소', '데이트', '산책로', '날씨', '친구', '코스', '...","['날씨좋을때 데이트하기 좋은곳', '산책하고 데이트하기 좋은 장소', '데이트하기...",긍정,0.986623,2
9,마포_경의선숲길공원,4,34,4_숲길_쓰레기_연남동_주민,"['숲길', '쓰레기', '연남동', '주민', '경의선', '사람', '겨울', ...",['경의선 숲길 산책하기 좋습니다 경의선 산책할 때 화장실가기는 힘듭니다 주변거주하...,중립,0.699922,1


In [24]:
eco_final_df['topic_weight'] = np.where(
    eco_final_df['topic_counts'] == 0,
    0.1,
    eco_final_df['topic_counts']
)
# 생태·경관 점수 계산
eco_final_df['eco_score'] = eco_final_df['sentiment_score'] * eco_final_df['Count']

print("생태·경관 점수 계산 완료!")
display(eco_final_df[['Park_nm', 'Name', 'sentiment_score', 'Count', 'eco_score']].head())

생태·경관 점수 계산 완료!


,Park_nm,Name,sentiment_score,Count,eco_score
0,용산_효창공원,1_나무_김구_기념관_효창,0.598557,64,38.307621
1,용산_효창공원,2_역사_심속_의미_오래,0.971144,43,41.759183
2,용산_효창공원,4_방문_아름답_시내_최고,0.616219,16,9.859497
3,종로_낙산공원,0_올라가_데이트_사람_예쁘,0.824299,581,478.917706
4,종로_낙산공원,1_카페_도심_올라가_주차장,0.942347,180,169.622437


In [25]:
# Park_nm에 '한강'이 포함된 데이터와 그렇지 않은 데이터로 분리
hangang_df = eco_final_df[eco_final_df['Park_nm'].str.contains('한강')].copy()
other_parks_df = eco_final_df[~eco_final_df['Park_nm'].str.contains('한강')].copy()

print(f"분석 대상 분리 완료:")
print(f"- 한강공원 관련 주제: {len(hangang_df)}개")
print(f"- 일반공원 관련 주제: {len(other_parks_df)}개")

분석 대상 분리 완료:
- 한강공원 관련 주제: 19개
- 일반공원 관련 주제: 157개


In [26]:
print("--- 한강공원 생태·경관성 순위 ---")

# 공원 이름('Park_nm')으로 그룹화하여 점수 합산
hangang_ranking = hangang_df.groupby('Park_nm')['eco_score'].sum().reset_index()

# z-score 계산
mean = hangang_ranking['eco_score'].mean()
std = hangang_ranking['eco_score'].std(ddof=0)
hangang_ranking['z'] = (hangang_ranking['eco_score'] - mean) / std

# z-score -> cdf 변환
hangang_ranking['cdf'] = norm.cdf(hangang_ranking['z'])

hangang_ranking['eco_score_std'] = hangang_ranking['cdf'] * 100

# 점수가 높은 순으로 정렬
hangang_ranking = hangang_ranking.sort_values(by='eco_score', ascending=False)

# 순위 부여
hangang_ranking['순위'] = range(1, len(hangang_ranking) + 1)

display(hangang_ranking[['순위', 'Park_nm', 'eco_score_std']])

output_file_path = "hangang_ranking_topic.csv"
hangang_ranking.to_csv(output_file_path, index=False, encoding='utf-8-sig')
print(f"Results saved to {output_file_path}")

--- 한강공원 생태·경관성 순위 ---


,순위,Park_nm,eco_score_std
4,1,한강_반포한강공원,95.960268
8,2,한강_잠실한강공원,93.267569
3,3,한강_망원한강공원,80.524383
9,4,한강_잠원한강공원,69.369733
5,5,한강_양화한강공원,28.325660
0,6,한강_광나루한강공원,27.807769
1,7,한강_난지한강공원,25.877336
6,8,한강_여의도한강공원,19.358658
2,9,한강_뚝섬한강공원,17.448238
7,10,한강_이촌한강공원,15.896241


Results saved to hangang_ranking_topic.csv


In [27]:
print("--- 일반공원 생태·경관성 순위 (개별) ---")

# 공원 이름('Park_nm')으로 그룹화하여 점수 합산
park_ranking = other_parks_df.groupby('Park_nm')['eco_score'].sum().reset_index()

# z-score 계산
mean = park_ranking['eco_score'].mean()
std = park_ranking['eco_score'].std(ddof=0)
park_ranking['z'] = (park_ranking['eco_score'] - mean) / std

# z-score -> cdf 변환
park_ranking['cdf'] = norm.cdf(park_ranking['z'])

park_ranking['eco_score_std'] = park_ranking['cdf'] * 100

# 점수가 높은 순으로 정렬
park_ranking = park_ranking.sort_values(by='eco_score', ascending=False)

# 순위 부여
park_ranking['순위'] = range(1, len(park_ranking) + 1)

display(park_ranking[['순위', 'Park_nm', 'eco_score_std']].head(10)) # 상위 10개만 출력

output_file_path = "nonhangang_ranking_topic.csv"
park_ranking.to_csv(output_file_path, index=False, encoding='utf-8-sig')
print(f"Results saved to {output_file_path}")

--- 일반공원 생태·경관성 순위 (개별) ---


,순위,Park_nm,eco_score_std
10,1,광진_어린이대공원,99.999994
17,2,동작_보라매공원,87.326200
6,3,강서_서울식물원,81.870184
32,4,종로_낙산공원,77.752169
29,5,영등포_여의도공원,75.711251
9,6,광진_아차산생태공원,68.924190
24,7,송파_석촌호수공원,65.201448
34,8,중구_남산공원,62.487966
0,9,etc_북한산국립공원,58.865743
15,10,도봉_서울창포원,56.969469


Results saved to nonhangang_ranking_topic.csv


In [28]:
print("--- 지역구별 공원 생태·경관성 순위 ---")

# 'Park_nm'에서 '_' 앞부분을 잘라 'District' (지역구) 컬럼 생성
other_parks_df['District'] = other_parks_df['Park_nm'].str.split('_').str[0]

# 지역구로 그룹화하여 점수 합산
district_ranking = other_parks_df.groupby('District')['eco_score'].sum().reset_index()

# z-score 계산
mean = district_ranking['eco_score'].mean()
std = district_ranking['eco_score'].std(ddof=0)
district_ranking['z'] = (district_ranking['eco_score'] - mean) / std

# z-score -> cdf 변환
district_ranking['cdf'] = norm.cdf(district_ranking['z'])

district_ranking['eco_score_std'] = district_ranking['cdf'] * 100

# 점수가 높은 순으로 정렬
district_ranking = district_ranking.sort_values(by='eco_score', ascending=False)

# 순위 부여
district_ranking['순위'] = range(1, len(district_ranking) + 1)

display(district_ranking[['순위', 'District', 'eco_score_std']])

output_file_path = "nonhangang_area_ranking_topic.csv"
district_ranking.to_csv(output_file_path, index=False, encoding='utf-8-sig')
print(f"Results saved to {output_file_path}")

--- 지역구별 공원 생태·경관성 순위 ---


,순위,District,eco_score_std
6,1,광진,99.998893
4,2,강서,74.859928
19,3,영등포,71.808862
22,4,중구,69.532084
17,5,송파,69.475871
12,6,동작,69.152568
21,7,종로,66.632124
18,8,양천,54.038365
0,9,etc,44.522577
10,10,도봉,43.193982


Results saved to nonhangang_area_ranking_topic.csv
